In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_hub as hub

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics  import classification_report, confusion_matrix

import os
os.listdir("../input")

In [ ]:
train = pd.read_csv('../input/train.csv', sep=',')
train.head()

In [ ]:
test = pd.read_csv('../input/test.csv')
test.head()

In [ ]:
sns.countplot(train['target']);

## Uniform data

In [ ]:
t1 = train[train['target']==1]
t0 = train[train['target']==0][:99999]

del train

In [ ]:
train = t1.append(t0)
train.head()

In [ ]:
train['length']  = train['question_text'].map(len)

In [ ]:
train['length'].describe()

## Data transformation

In [ ]:
import nltk, re, string
from keras.preprocessing import sequence, text
def text_preprocess(text):
    text = text.lower()
    text = re.sub('[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]', ' ', text)
    return text

In [ ]:
train['clean_text'] = train['question_text'].map(text_preprocess)
test['clean_text'] = test['question_text'].map(text_preprocess)

train['clean_text'].head()

In [ ]:
max_features = 2000
tokenizer = text.Tokenizer(num_words=max_features, lower=True)

In [ ]:
train['clean_text'][:10]

In [ ]:
tokenizer.fit_on_texts(train['clean_text'])

In [ ]:
X_train = tokenizer.texts_to_sequences(train['clean_text'])

In [ ]:
X_test = tokenizer.texts_to_sequences(test['clean_text'])

##  Model creation

In [ ]:
def encode(x):
    return keras.utils.to_categorical(x)

def decode(x):
    return np.argmax(x, axis=1)

In [ ]:
x_train = list(X_train)
y_train = list(train['target'])

In [ ]:
y_train = encode(y_train)
y_train

## Model Training

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Lambda, Input, Embedding, Bidirectional, LSTM
import keras.backend as K

from keras.preprocessing.text import text_to_word_sequence
import tensorflow as tf

In [ ]:
max_features = 2196

In [ ]:
model = Sequential()
hidden_size = 128
# model.add(Dense(activation='relu'))
model.add(Embedding(max_features, hidden_size))
model.add(Bidirectional(LSTM(hidden_size, activation='elu', dropout=0.4, recurrent_dropout=0.2)))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
x_train = sequence.pad_sequences(X_train, maxlen=71, padding='post', truncating='post')

In [ ]:
model.fit(x_train, y_train, batch_size=128, epochs=10, shuffle=True, validation_split=0.1)

In [ ]:
score = model.evaluate(x=x_train[25:10025], y=y_train[25:10025], batch_size=16)

In [ ]:
score

## Metrics Score

In [ ]:
# y_pr = decode(y_preds)

In [ ]:
# c = confusion_matrix(np.argmax(y_train, axis=1), y_pr)
# sns.heatmap(c, annot=True);

## Model Prediction Test

In [ ]:
x_test = list(X_test)

In [ ]:
x_test = sequence.pad_sequences(x_test, maxlen=71, padding='post', truncating='post')

In [ ]:
test_preds = model.predict(x_test, batch_size=64)

In [ ]:
test_preds = np.argmax(test_preds, axis=1)
test_preds

## Submission 

In [ ]:
sub = pd.read_csv('../input/sample_submission.csv')
sub.head()

In [ ]:
sub['prediction'] = test_preds

In [ ]:
sns.countplot(test_preds);

In [ ]:
sub.to_csv('submission.csv', index=False)